In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Armado de features**

In [220]:
window = 1

In [221]:
auctions = pd.read_pickle('data/auctions_w'+str(window)+'.pkl')
clicks_w1 = pd.read_pickle('data/clicks_w'+str(window)+'.pkl')
installs = pd.read_pickle('data/installs_w'+str(window)+'.pkl')
events_w1 = pd.read_pickle('data/events_w'+str(window)+'.pkl')
if window <= 4:
    auctions_label = pd.read_pickle('data/auctions_w'+str(window)+'_label.pkl')
    installs_label = pd.read_pickle('data/installs_w'+str(window)+'_label.pkl')

In [222]:
time_windows = [('2019-04-18 00:00:00.000000', '2019-04-21 00:00:00.000000'), ('2019-04-19 00:00:00.000000', '2019-04-22 00:00:00.000000'), 
                ('2019-04-20 00:00:00.000000', '2019-04-23 00:00:00.000000'), ('2019-04-21 00:00:00.000000', '2019-04-24 00:00:00.000000'), 
                ('2019-04-24 00:00:00.000000', '2019-04-27 00:00:00.000000')]
time_labels = [('2019-04-21 00:00:00.000000', '2019-04-24 00:00:00.000000'), ('2019-04-22 00:00:00.000000', '2019-04-25 00:00:00.000000'), 
               ('2019-04-23 00:00:00.000000', '2019-04-26 00:00:00.000000'), ('2019-04-24 00:00:00.000000', '2019-04-27 00:00:00.000000')]
days = [18, 19, 20, 21, 24]

In [206]:
gb = auctions.groupby('ref_hash')
auctions['n_auctions'] = gb['date'].transform('count')
auctions['last_auction'] = gb['date'].transform('max')
auctions['first_auction'] = gb['date'].transform('min')
auctions['diff_auctions'] = (auctions['last_auction'] - auctions['first_auction']).dt.total_seconds()
auctions['mean_time_auction'] = 0
auctions.loc[auctions['n_auctions'] > 1, 'mean_time_auction'] = ((auctions['last_auction'] - auctions['first_auction'])/ \
                                 (auctions['n_auctions'] -1)).dt.total_seconds()
auctions['first_auction_sec'] = (auctions['first_auction'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
auctions['last_auction_sec'] = (auctions['last_auction'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
auctions['ref_type_id_1'] = auctions['ref_type_id'].apply(lambda x: 1 if x==1 else 0)
auctions['ref_type_id_7'] = auctions['ref_type_id'].apply(lambda x: 1 if x==7 else 0)
auctions = pd.get_dummies(auctions, columns=['source_id'])
auctions['day'] = (auctions['date'].dt.day) - days[window-1]
gb = auctions.groupby('ref_hash')
auctions['mean_day'] = gb['day'].transform('mean')
auctions = pd.get_dummies(auctions, columns=['day'])
gb = auctions.groupby('ref_hash')
auctions['day_0'] = gb['day_0'].transform('sum')
auctions['day_1'] = gb['day_1'].transform('sum')
auctions['day_2'] = gb['day_2'].transform('sum')

In [207]:
auctions.drop_duplicates(subset='ref_hash', inplace=True)
auctions = auctions[['ref_hash', 'n_auctions', 'diff_auctions', 'mean_time_auction', 
                     'first_auction_sec', 'last_auction_sec', 'ref_type_id_1',
                     'ref_type_id_7', 'source_id_0', 'source_id_1', 'source_id_2',
                     'source_id_3', 'source_id_4', 'source_id_5', 'source_id_6',
                     'source_id_7', 'source_id_8', 'source_id_9', 'mean_day', 
                     'day_0', 'day_1', 'day_2']]

In [208]:
gb = clicks_w1.groupby('ref_hash')
clicks_w1['n_clicks'] = gb['created'].transform('count')
clicks_w1['last_click'] = gb['created'].transform('max')
clicks_w1['first_click'] = gb['created'].transform('min')
clicks_w1['diff_clicks'] = (clicks_w1['last_click'] - clicks_w1['first_click']).dt.total_seconds()
clicks_w1['mean_time_click'] = 0
clicks_w1.loc[clicks_w1['n_clicks'] > 1, 'mean_time_click'] = ((clicks_w1['last_click'] - clicks_w1['first_click'])/ \
                                 (clicks_w1['n_clicks'] -1)).dt.total_seconds()
clicks_w1['first_click_sec'] = (clicks_w1['first_click'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
clicks_w1['last_click_sec'] = (clicks_w1['last_click'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
clicks_w1['wifi_connection'] = clicks_w1['wifi_connection'].map({True: 1, False: 0})
clicks_w1['timeToClick_mean'] = clicks_w1.groupby('ref_hash')['timeToClick'].transform('mean')
clicks_w1.loc[clicks_w1.touchX == 'Infinity', 'touchX'] = 1
clicks_w1.loc[clicks_w1.touchY == 'Infinity', 'touchY'] = 1
clicks_w1["touchX"] = pd.to_numeric(clicks_w1["touchX"])
clicks_w1["touchY"] = pd.to_numeric(clicks_w1["touchY"])
clicks_w1['touchX_mean'] = clicks_w1.groupby('ref_hash')['touchX'].transform('mean')
clicks_w1['touchY_mean'] = clicks_w1.groupby('ref_hash')['touchY'].transform('mean')
clicks_w1['latitude_mean'] = clicks_w1.groupby('ref_hash')['latitude'].transform('mean')
clicks_w1['longitude_mean'] = clicks_w1.groupby('ref_hash')['longitude'].transform('mean')
clicks_w1['timeToClick_mean'] = clicks_w1['timeToClick_mean'].fillna(clicks_w1['timeToClick_mean'].mean())
clicks_w1['touchX_mean'] = clicks_w1['touchX_mean'].fillna(clicks_w1['touchX_mean'].mean())
clicks_w1['touchY_mean'] = clicks_w1['touchY_mean'].fillna(clicks_w1['touchY_mean'].mean())

In [209]:
clicks_w1.drop_duplicates(subset='ref_hash', inplace=True)
clicks_w1 = clicks_w1[['ref_hash', 'touchX_mean', 'touchY_mean', 'timeToClick_mean', 'latitude_mean', 'longitude_mean', 
                       'n_clicks', 'wifi_connection', 'first_click_sec', 'last_click_sec', 'diff_clicks', 'mean_time_click']]

In [210]:
events_w1.drop(columns=['event_uuid', 'ip_address', 'index', 'device_countrycode'], inplace=True)
events_w1['n_events'] = events_w1.groupby('ref_hash')['date'].transform('count')
events_w1['attributed'] = events_w1['attributed'].map({True: 1, False: 0})
events_w1['wifi'] = events_w1['wifi'].map({True: 1, False: 0})
events_w1['attributed_events_mean'] = events_w1.groupby('ref_hash')['attributed'].transform('mean')
events_w1['wifi_events_mean'] = events_w1.groupby('ref_hash')['wifi'].transform('mean')
events_w1['first_event'] = events_w1.groupby('ref_hash')['date'].transform('min')
events_w1['last_event'] = events_w1.groupby('ref_hash')['date'].transform('max')
events_w1['diff_events'] = (events_w1['last_event'] - events_w1['first_event']).dt.total_seconds()
events_w1['mean_time_events'] = 0
events_w1.loc[events_w1['n_events'] > 1, 'mean_time_events'] = ((events_w1['last_event'] - events_w1['first_event'])/ \
                                 (events_w1['n_events'] -1)).dt.total_seconds()
events_w1['first_event_sec'] = (events_w1['first_event'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
events_w1['last_event_sec'] = (events_w1['last_event'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()

In [211]:
events_w1.drop_duplicates(subset='ref_hash', inplace=True)
events_w1 = events_w1[['ref_hash', 'first_event_sec', 'last_event_sec', 'n_events', 'attributed_events_mean', 'wifi_events_mean', 'diff_events', 'mean_time_events']]

In [212]:
dtypes_dict = {'created':'str',  'ref_type': 'category', 'device_brand': np.float32, \
                'device_countrycode': 'category', 'device_language': 'category', \
                 'kind': 'str', 'user_agent': 'str'}

installs = pd.read_csv('data/installs.csv.gzip', dtype=dtypes_dict, low_memory=False)
installs = installs.drop(columns=['device_language', 'device_countrycode', 'ip_address', 'event_uuid', 'device_brand', \
                         'device_model', 'click_hash', 'session_user_agent'])
installs['created'] = pd.to_datetime(installs['created'], format='%Y-%m-%d %H:%M:%S')
installs['wifi_installs'] = installs['wifi'].map({True: 1, False: 0})
installs['attributed'] = installs['attributed'].map({True: 1, False: 0})
installs['implicit'] = installs['implicit'].map({True: 1, False: 0})
gb = installs.groupby('ref_hash')
installs['wifi_installs_mean'] = gb['wifi_installs'].transform('mean')
installs['attributed_installs_mean'] = gb['attributed'].transform('mean')
installs['n_installs'] = gb['created'].transform('count')
installs['last_install'] = gb['created'].transform('max')
installs['first_install'] = gb['created'].transform('min')
installs['diff_installs'] = (installs['last_install'] - installs['first_install']).dt.total_seconds()
installs['mean_time_install'] = 0
installs.loc[installs['n_installs'] > 1, 'mean_time_install'] = ((installs['last_install'] - installs['first_install'])/ \
                                 (installs['n_installs'] -1)).dt.total_seconds()
installs['first_install_sec'] = (installs['first_install'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()
installs['last_install_sec'] = (installs['last_install'] - pd.Timestamp(time_windows[window-1][0])).dt.total_seconds()

In [213]:
installs.drop_duplicates(subset='ref_hash', inplace=True)
installs = installs[['ref_hash', 'first_install_sec', 'last_install_sec', 'n_installs', 'attributed_installs_mean', 'wifi_installs_mean', 'diff_installs', 'mean_time_install']]

In [214]:
model = pd.merge(auctions, clicks_w1, on='ref_hash', how='left')

In [215]:
model = pd.merge(model, installs, on='ref_hash', how='outer')

In [216]:
model = pd.merge(model, events_w1, on='ref_hash', how='left')

In [217]:
model.shape

(554681, 47)

In [218]:
if window <= 4:
    model = pd.merge(model, auctions_label, on='ref_hash', how='left')
    model = pd.merge(model, installs_label, on='ref_hash', how='left')
    model['time_appearence'] = model['time_appearence'].fillna(259200)
    model['time_appearence_install'] = model['time_appearence_install'].fillna(259200)

In [219]:
model.drop_duplicates('ref_hash', inplace=True)
model.to_pickle('data/model/model3_w'+str(window)+'.pkl')